#Pothole Detection

#Extract Dataset

In [ ]:
import zipfile

zip_path = "/content/PotholeDataset.zip"
extract_path = "/content/Pothole_Data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted to:", extract_path)


BadZipFile: File is not a zip file

#Import Libraries

In [ ]:
import numpy as np
import cv2
import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Load Dataset

In [ ]:
size = 150   # image size

def load_images(path):
    images = []
    files = glob.glob(path + "/*")
    for f in files:
        img = cv2.imread(f, 0)
        img = cv2.resize(img, (size, size))
        images.append(img)
    return np.asarray(images)


#Load images

In [ ]:
train_pothole = load_images("/content/Pothole_Data/PotholeDataset/train/Pothole")
train_plain   = load_images("/content/Pothole_Data/PotholeDataset/train/Plain")

test_pothole  = load_images("/content/Pothole_Data/PotholeDataset/test/Pothole")
test_plain    = load_images("/content/Pothole_Data/PotholeDataset/test/Plain")

#Prepare Training + Test Sets

In [ ]:
X_train = np.concatenate([train_pothole, train_plain])
X_test  = np.concatenate([test_pothole, test_plain])

y_train = np.concatenate([
    np.ones(len(train_pothole)),  # pothole = 1
    np.zeros(len(train_plain))    # plain = 0
])

y_test = np.concatenate([
    np.ones(len(test_pothole)),
    np.zeros(len(test_plain))
])

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

X_train = X_train.reshape(-1, size, size, 1) / 255.0
X_test  = X_test.reshape(-1, size, size, 1) / 255.0

y_train = to_categorical(y_train, 2)
y_test  = to_categorical(y_test, 2)

print("Train:", X_train.shape)
print("Test:",  X_test.shape)


#Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

datagen.fit(X_train)


#Build Strong CNN Model

In [ ]:
def build_model():
    model = Sequential()

    model.add(Conv2D(32,(3,3),activation="relu",padding="same",input_shape=(150,150,1)))
    model.add(MaxPooling2D(2,2))

    model.add(Conv2D(64,(3,3),activation="relu",padding="same"))
    model.add(MaxPooling2D(2,2))

    model.add(Conv2D(128,(3,3),activation="relu",padding="same"))
    model.add(MaxPooling2D(2,2))

    model.add(Conv2D(256,(3,3),activation="relu",padding="same"))
    model.add(MaxPooling2D(2,2))

    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))

    model.add(Dense(2, activation="softmax"))
    return model

model = build_model()
model.compile(optimizer=Adam(0.0004), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


#Train-Validation Split

In [ ]:
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, y_train, test_size=0.2, shuffle=True
)


#TRAIN THE MODEL

In [ ]:
history = model.fit(
    datagen.flow(X_train2, y_train2, batch_size=32),
    epochs=30,
    validation_data=(X_val, y_val)
)


#Evaluate on Test Set

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc * 100, "%")

#Save Model

In [ ]:
model.save("pothole_cnn_model.h5")
print("Model saved as pothole_cnn_model.h5")

#Confusion Matrix + Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Predict
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Plain','Pothole'],
            yticklabels=['Plain','Pothole'])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

print("\nClassification Report:\n")
print(classification_report(y_true_labels, y_pred_labels, target_names=['Plain','Pothole']))

NameError: name 'model' is not defined

#Accuracy & Loss Graph

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))

# Accuracy graph
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss graph
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

#Real-time Image Prediction With Parameters

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow

# Load your CNN model
model = load_model("pothole_cnn_model.h5")
size = 150


# ---------------------------
# AREA & DEPTH CALCULATION
# ---------------------------
def calculate_area_depth(gray_img):
    """
    Returns:
        area (0.0 to 1.0)
        depth (0.0 to 1.0)
    """

    # Threshold to isolate dark pothole-like regions
    _, th = cv2.threshold(gray_img, 120, 255, cv2.THRESH_BINARY_INV)

    total_pixels = gray_img.size
    dark_pixels = np.sum(th == 255)

    # AREA ESTIMATE (ratio)
    area_ratio = dark_pixels / total_pixels

    # DEPTH ESTIMATE (inverse brightness of dark regions)
    if dark_pixels == 0:
        depth_ratio = 0
    else:
        dark_region = gray_img[th == 255]
        avg_dark = np.mean(dark_region)
        depth_ratio = 1 - (avg_dark / 255)   # 0 (bright) → 1 (dark)

    return round(area_ratio, 3), round(depth_ratio, 3)


# ---------------------------
# PROBABILITY LOGIC
# ---------------------------
def evaluate_pothole(area, depth):
    """
    Decide pothole severity based on area & depth.
    """

    # If nothing detected → plain road
    if area == 0 or depth == 0:
        probability = 0
        message = "✔ Plain road — safe to drive."
        return probability, message

    # Probability formula
    probability = (area * 0.6 + depth * 0.4) * 100
    probability = round(probability, 2)

    # Decision
    if probability >= 50:
        message = "❗ Reduce speed immediately — major pothole detected."
    else:
        message = "⚠ Drive slow — mild pothole ahead."

    return probability, message



# ---------------------------
# MAIN PREDICTION FUNCTION
# ---------------------------
def predict_image(image_path):

    img_color = cv2.imread(image_path)
    if img_color is None:
        print("❌ ERROR: Image not found.")
        return

    img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)

    # CNN classification
    resized = cv2.resize(img_gray, (size, size))
    cnn_input = resized.reshape(1, size, size, 1) / 255.0
    pred = model.predict(cnn_input)

    class_id = np.argmax(pred)
    confidence = float(np.max(pred))

    label = "normal" if class_id == 0 else "pothole"


    # ----------------------------------------------------------
    # ✔ FIXED: If CNN predicts NORMAL → IGNORE AREA + DEPTH
    # ----------------------------------------------------------
    if label == "normal":
        print("\n=======================")
        print("      IMAGE PREVIEW")
        print("=======================\n")
        cv2_imshow(img_color)

        print("\n=======================")
        print("      RESULT")
        print("=======================\n")

        print("Prediction: NORMAL")
        print(f"CNN Confidence: {round(confidence * 100, 2)} %")
        print("Estimated Area: 0.0")
        print("Estimated Depth: 0.0")
        print("Probability: 0 %")
        print("\nMessage: ✔ Normal road — you can drive normally.")
        print("\n=======================\n")
        return
    # ----------------------------------------------------------


    # For POTHOLE → calculate area & depth
    area, depth = calculate_area_depth(img_gray)
    probability, message = evaluate_pothole(area, depth)

    # ----- OUTPUT -----
    print("\n=======================")
    print("      IMAGE PREVIEW")
    print("=======================\n")
    cv2_imshow(img_color)

    print("\n=======================")
    print("      RESULT")
    print("=======================\n")

    print(f"Prediction: POTHOLE")
    print(f"CNN Confidence: {round(confidence * 100, 2)} %")
    print(f"Estimated Area: {area}")
    print(f"Estimated Depth: {depth}")
    print(f"Probability: {probability} %")
    print("\nMessage:", message)
    print("\n=======================\n")

In [ ]:
predict_image("/content/Pothole5.jpg")

In [ ]:
predict_image("/content/Pothole6.jpg")

In [ ]:
predict_image("/content/Plain1.jpg")

In [ ]:
# Save the model in .keras format
model.save("pothole_cnn_model.keras")
print("Model saved as pothole_cnn_model.keras")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.build((None, 150, 150, 1))   # Ensure new input spec
model.save("pothole_cnn_model.keras", save_format="keras")
print("Model Saved Successfully!")
